# T81 Orientation Notebook — 00

This notebook is an **executable orientation** to the `t81-foundation` repository.

It is designed to:

1. Verify you are running inside the repo.
2. Provide a small **balanced ternary** and **base-81** playground in pure Python.
3. Point you into the canonical **spec documents** and **C++ headers**.

> This notebook is pedagogical.  
> Canonical behavior, performance, and semantics are defined by the C++ implementation and the specifications in `spec/`.


## 0. How to Use This Notebook

Recommended usage:

1. Clone the repo (if you have not already):

   ```bash
   git clone https://github.com/t81dev/t81-foundation.git
   cd t81-foundation
   ```

2. Start Jupyter from the repo root:

   ```bash
   jupyter lab
   # or
   jupyter notebook
   ```

3. Open: `notebooks/00_t81_orientation.ipynb`.

This notebook assumes:

- You are in the **repository root** (where `CMakeLists.txt` and `spec/` live).
- You have a working Python 3 environment.
- You treat this notebook as an **on-ramp**, not as a replacement for the formal specs in `spec/`.


In [ ]:
import os
from pathlib import Path

repo_root = Path(".").resolve()
expected_files = ["CMakeLists.txt", "spec", "include", "src"]

print("Detected working directory:")
print(repo_root)
print()

missing = [p for p in expected_files if not (repo_root / p).exists()]

if missing:
    print("⚠️ This does not look like the t81-foundation repo root.")
    print("Missing:", ", ".join(missing))
    print("\nPlease `cd` into the repository root before running this notebook.")
else:
    print("✅ This looks like the t81-foundation repository root.")
    print("Found:", ", ".join(expected_files))


## 1. Build & Test (Out-of-Notebook)

T81 is primarily a **C++20** project with a formal spec in `spec/`.

You can build and run tests from a shell:

```bash
cmake -S . -B build
cmake --build build -j
ctest --test-dir build -R "t81_"
```

This notebook does **not** automatically build the project, but it assumes:

- The codebase is at least buildable.
- You treat the Python snippets here as **conceptual mirrors** of the canonical C++ behavior.

For authoritative definitions, see:

- `spec/t81-overview.md`
- `spec/t81-data-types.md`
- `spec/t81vm-spec.md`
- `include/t81/*.hpp`


## 2. Balanced Ternary — Conceptual Playground

T81 uses **balanced ternary** at its core: digits from \(\{-1, 0, +1\}\).

A common symbolic convention is:

- `-1 → "-"` (or sometimes `T`)
- `0  → "0"`
- `+1 → "+"` (or sometimes `1`)

In this section, we implement a **simple, non-optimized** balanced ternary encoder/decoder in Python:

- `int → balanced ternary string`
- `balanced ternary string → int`

This is for intuition and experimentation; it is **not** the canonical implementation used by the C++ core.


In [ ]:
from typing import List

# We use "-" for -1, "0" for 0, "+" for +1.
BT_DIGITS = (-1, 0, 1)
BT_SYMBOLS = {
    -1: "-",
    0: "0",
    1: "+",
}
BT_SYMBOL_TO_VALUE = {v: k for k, v in BT_SYMBOLS.items()}


def int_to_balanced_ternary(n: int) -> str:
    """
    Convert a Python int to a balanced ternary string using symbols '-', '0', '+'.

    This is a simple reference implementation for educational purposes.
    """
    if n == 0:
        return "0"

    digits: List[int] = []
    x = n

    while x != 0:
        x, remainder = divmod(x, 3)

        # Convert [0, 1, 2] remainder into [-1, 0, +1] balanced ternary digit.
        if remainder == 2:
            # 2 in regular ternary is '-1' with a carry of +1.
            remainder = -1
            x += 1

        digits.append(remainder)

    # Most significant digit last in the list → reverse for string
    symbols = [BT_SYMBOLS[d] for d in reversed(digits)]
    return "".join(symbols)


def balanced_ternary_to_int(s: str) -> int:
    """
    Convert a balanced ternary string ('-', '0', '+') back to a Python int.
    """
    value = 0
    for ch in s:
        if ch not in BT_SYMBOL_TO_VALUE:
            raise ValueError(f"Invalid balanced ternary digit: {ch!r}")
        d = BT_SYMBOL_TO_VALUE[ch]
        value = value * 3 + d
    return value


# Quick sanity checks
for n in range(-10, 11):
    bt = int_to_balanced_ternary(n)
    back = balanced_ternary_to_int(bt)
    print(f"{n:3d} → {bt:>5s} → {back:3d}")


### 2.1. Balanced Ternary Arithmetic (Toy Example)

Below is a small helper for **addition** in balanced ternary space:

1. Convert strings to integers.
2. Add.
3. Convert back to balanced ternary.

This keeps the code simple and readable.  
A more sophisticated implementation would work directly on ternary digits, mirroring the VM’s behavior.


In [ ]:
def bt_add(a: str, b: str) -> str:
    """
    Add two balanced ternary numbers given as strings ('-', '0', '+'),
    returning a balanced ternary string.
    """
    ia = balanced_ternary_to_int(a)
    ib = balanced_ternary_to_int(b)
    return int_to_balanced_ternary(ia + ib)


# Example usage:
a = int_to_balanced_ternary(5)
b = int_to_balanced_ternary(-3)
c = bt_add(a, b)

print("a =", a, "=", balanced_ternary_to_int(a))
print("b =", b, "=", balanced_ternary_to_int(b))
print("a + b =", c, "=", balanced_ternary_to_int(c))


## 3. Base-81 Encoding — Conceptual Mirror

T81 uses **base-81** encodings with a fixed alphabet (see `include/t81/hash/base81.hpp`).

In this section, we create a simple **reference** encoder/decoder:

- `bytes → base-81 string`
- `base-81 string → bytes`

Important:

- This implementation is **for demonstration only**.
- Canonical behavior, edge cases, and backward-compatibility (e.g., `"b81:"` prefixes) are defined in the actual C++ implementation.

You can use this section to:

- Experiment with how data compresses/expands under base-81.
- Verify round-trips for small payloads.


In [ ]:
# Example alphabet: must have length 81.
# For orientation only; keep in sync conceptually with the C++ alphabet.
BASE81_ALPHABET = (
    "0123456789"              # 10
    "ABCDEFGHIJKLMNOPQRSTUVWXYZ"  # 26 → 36 total
    "abcdefghijklmnopqrstuvwxyz"  # 26 → 62 total
    "!@#$%^&*()-_=+[]{};:,./?"    # 19 → 81 total
)

assert len(BASE81_ALPHABET) == 81, "Base-81 alphabet must have length 81."


def encode_base81(data: bytes) -> str:
    """
    Naive base-81 encoder for demonstration.

    - Interprets the input as a big-endian integer.
    - Emits base-81 digits using BASE81_ALPHABET.
    - Does not implement any canonical length/metadata behavior.
    """
    if not data:
        return BASE81_ALPHABET[0]

    n = int.from_bytes(data, "big")

    if n == 0:
        return BASE81_ALPHABET[0]

    digits = []
    while n > 0:
        n, r = divmod(n, 81)
        digits.append(BASE81_ALPHABET[r])

    return "".join(reversed(digits))


def decode_base81(s: str) -> bytes:
    """
    Inverse of encode_base81 for demonstration.

    - Reconstructs the big integer.
    - Converts back to bytes with minimal length.
    """
    if not s:
        raise ValueError("Empty base-81 string.")

    n = 0
    for ch in s:
        idx = BASE81_ALPHABET.find(ch)
        if idx == -1:
            raise ValueError(f"Invalid base-81 character: {ch!r}")
        n = n * 81 + idx

    # Compute the minimal byte length needed.
    if n == 0:
        return b"\x00"
    length = (n.bit_length() + 7) // 8
    return n.to_bytes(length, "big")


# Quick demonstration
sample = b"T81 foundation"
encoded = encode_base81(sample)
decoded = decode_base81(encoded)

print("Sample bytes:   ", sample)
print("Base-81 encoded:", encoded)
print("Round-trip OK?: ", decoded == sample)


## 4. T81 Stack — Conceptual Diagram

The T81 stack spans from numeric substrate → VM → language → cognitive/ethical layers.

A simplified orientation:

```text
+-----------------------------------------------------------+
| Cognitive Tiers / Axion (Ethical Supervisor)              |
+-----------------------------------------------------------+
| T81Lang — High-level language, symbolic + numeric         |
+-----------------------------------------------------------+
| TISC — Ternary Instruction Set (abstract machine spec)    |
+-----------------------------------------------------------+
| T81 VM (HanoiVM lineage) — Deterministic execution        |
+-----------------------------------------------------------+
| T81 / T243 / T729 Types — BigInt, Fraction, Tensor        |
+-----------------------------------------------------------+
| Balanced Ternary & Base-81 Numeric Substrate              |
+-----------------------------------------------------------+
| Future Ternary Hardware / Emulated Targets                |
+-----------------------------------------------------------+
```

For **formal definitions**, see:

- `spec/t81-overview.md`
- `spec/t81-data-types.md`
- `spec/t81vm-spec.md`
- `spec/t81lang-spec.md`
- `spec/axion-kernel.md` (or equivalent Axion spec file, if present)


## 5. Where to Go Next in the Repo

Use this notebook as a **map**, not as the territory.

### If you care about numerical substrate

- Read: `spec/t81-data-types.md`
- Browse: `include/t81/bigint.hpp`, `include/t81/fraction.hpp`, `include/t81/tensor.hpp` and friends.
- Compare their semantics to the simplified Python examples in this notebook.

### If you care about the VM and instruction set

- Read: `spec/t81vm-spec.md` and `spec/tisc-spec.md` (names may vary slightly depending on current spec layout).
- Inspect legacy HanoiVM sources in:
  - `legacy/hanoivm/src/hanoivm_core/`
- Then look at the emerging C++ IR / VM headers under `include/t81/ir/` and associated `src/` files.

### If you care about Axion and cognitive layers

- Explore: `spec/axion-kernel.md`, `spec/cognitive-tiers.md`, and any related documents.
- Treat those as the **constitutional layer** that governs how computation is used, not just how it executes.

---

## 6. Possible Next Notebooks

You might later add:

- `01_balanced_ternary_playground.ipynb`  
  Deep dive on arithmetic, promotion rules (T81 → T243 → T729), and tensor/BigInt interactions.

- `02_t81vm_trace_walkthrough.ipynb`  
  A small TISC program, executed via a Python-friendly interface, with step-by-step VM state traces.

For now, this `00_t81_orientation.ipynb` is meant to be:

- A **first contact** for new readers.
- A quick **sanity check** that the ideas are concrete and executable.
